In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

folder_path = "/Users/mukuldixit/Desktop/projects/SmartEye/data/raw/Train/Abuse"

video_id = "Abuse001"
frames = [
    f for f in os.listdir(folder_path)
    if f.startswith(video_id) and f.endswith((".png", ".jpg"))
]

frame_list = []

for f in frames:
    frame_number = int(f.split("_x264_")[1].split(".")[0])
    frame_list.append((frame_number, f))

frame_list.sort(key=lambda x: x[0])

scores = []

prev = cv2.imread(os.path.join(folder_path, frame_list[0][1]))
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

for frame_number, filename in frame_list[1:]:
    frame = cv2.imread(os.path.join(folder_path, filename))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(prev_gray, gray)
    score = np.mean(diff)
    scores.append(score)

    prev_gray = gray

scores = np.array(scores)

mean = np.mean(scores)
std = np.std(scores)
threshold = mean + 2.5 * std

spike_indices = np.where(scores > threshold)[0]

print("Spike indices:", spike_indices[:5])

# Plot movement curve
plt.figure(figsize=(12,5))
plt.plot(scores)
plt.axhline(threshold, color='r', linestyle='--')
plt.title("Movement Score per Frame")
plt.xlabel("Frame Index")
plt.ylabel("Movement Score")
plt.show()


In [ ]:
if len(spike_indices) > 0:
    spike_idx = spike_indices[0]
    spike_frame_name = frame_list[spike_idx][1]

    spike_frame = cv2.imread(os.path.join(folder_path, spike_frame_name))
    cv2.imshow("Spike Frame", spike_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print("Spike Frame Number:", frame_list[spike_idx][0])
